In [1]:
import os, sys, time, pickle
from pathlib import Path
from collections import Counter

PROJECT_DIR = Path(sys.path[0])/".."
DATA_DIR = PROJECT_DIR/"data"
SRC_DIR = PROJECT_DIR/"src"
DEPS_DIR = PROJECT_DIR/"deps"

sys.path.append(str(SRC_DIR))
sys.path.append(str(DEPS_DIR))

%pylab inline
%config InlineBackend.figure_format='retina'

%load_ext autoreload
%autoreload 2

from tqdm import tqdm

import torch
from torch.utils.data import Dataset, DataLoader
from fairseq_wav2vec import Wav2Vec2Config, Wav2Vec2Model
import pandas as pd

from mugen_train import musicDataset

%pylab is deprecated, use %matplotlib inline and import the required libraries.
Populating the interactive namespace from numpy and matplotlib


/home/keld/miniconda3/envs/mugen_ml/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


data

In [2]:
top50_genre_samples_df = pd.read_csv("/n1Tb/sc_mp3_top50_genre_samples.tsv_gz", compression='gzip', sep='\t')

n_size=1e5
cut=100000
ds = musicDataset(top50_genre_samples_df, n_size=n_size, cut=cut)

train_loader = DataLoader(
    ds, batch_size=6, shuffle=True, 
    collate_fn=lambda x: torch.concat([item for item in x if item is not None and item.shape[1] >= cut])
)

In [3]:
model = Wav2Vec2Model(Wav2Vec2Config)
model = model.to("cuda")

optimizer = torch.optim.Adam(model.parameters(), lr=0.005, betas=(0.9,0.98), eps=1e-06, weight_decay=0.01, fused=True)

In [6]:
%%timeit -n 1 -r 1
for b in train_loader:
    break

69 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [7]:
optimizer.zero_grad()

In [11]:
%%timeit -n 1 -r 1
for i, b in enumerate(train_loader):
    outputs = model(b)
    if i>3:
        break


1.02 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [4]:
# train

def train(model, optimizer, train_loader):
    model.train()
    running_loss = 0.0
    for i, batch in tqdm(enumerate(train_loader, 0), total=len(train_loader)):
        optimizer.zero_grad()
        outputs = model(batch)
        loss = outputs["features_pen"]
        loss.backward()
        optimizer.step()
        running_loss += loss.to("cpu").item()
    return running_loss / len(train_loader)

In [5]:
train(model, optimizer, train_loader)

  2%|▉                                                        | 144/8334 [00:45<42:47,  3.19it/s]


KeyboardInterrupt: 

In [7]:
%debug

> /home/keld/miniconda3/envs/mugen_ml/lib/python3.9/site-packages/miniaudio.py(567)mp3_read_f32()
    565         memory = lib.drmp3_open_memory_and_read_pcm_frames_f32(data, len(data), config, num_frames, ffi.NULL)
    566         if not memory:
--> 567             raise DecodeError("cannot load/decode data")
    568         try:
    569             samples = array.array('f')



ipdb>  up


> /home/keld/projects/music_gen/mugen_ml/src/audio_lib.py(32)decode()
     30         with open(self.fp, "rb") as f:
     31             f.seek(offset, 0)
---> 32             decoded = mp3_read_f32(f.read(n_size))
     33         self.decoded = decoded
     34         self.sample_rate = decoded.sample_rate



ipdb>  n_size


100000


ipdb>  q


In [49]:
for epoch in range(10):
    train_loss = train(model, optimizer, criterion, train_loader)
    print(f"Epoch {epoch+1}, train loss: {train_loss:.3f}")


In [66]:
for i, batch in enumerate(top50_dataloader):
    break

In [18]:
out = model.forward(sample_bs)

In [160]:
sample_bs.shape

torch.Size([32, 120000])

In [144]:

d.shape

torch.Size([2, 273024])